Read the Docs
https://nominatim.org/release-docs/develop/

In [81]:
import requests
import pandas as pd
import numpy as np
import sys
from geopy import distance

In [2]:
url = 'https://nominatim.openstreetmap.org/search?'

In [14]:
query = {
    city = 'Bethesda',
    state = 'Maryland',
    country = 'United States'
}

SyntaxError: invalid syntax (<ipython-input-14-8c4ac5bcf717>, line 2)

In [57]:
limit = 1
results = requests.get(url+'q=Clear Lake,Washington,United States&format=json&limit={0}'.format(limit))

In [58]:
js = results.json()

In [59]:
for i in js:
    for j in i:
        if (j == 'lat') | (j == 'lon'):
            print(j)
            print(i.get(j))

lat
48.4551255
lon
-122.24035880594406


In [ ]:
"""
Pseudo Code:
1) For every address, try to access API; if error skip to next row and insert error row and error value
    - check for status code before moving to next value
2) If request successful, attempt to access lon and latitude.  If nothing, skip to next row; if multiple results, insert first results
"""

In [78]:
def get_latLong(city,state,country,limit=1):
    try:
        results = requests.get('https://nominatim.openstreetmap.org/search?q={0},{1},{2}&format=json&limit={3}'.format(city,state,country,limit))
        req = results.json()[0]
        return req.get('lat'), req.get('lon')
    except:
        return sys.exc_info()[0]

In [129]:
myData = []

In [115]:
dc = get_latLong('Washington','District of Columbia','United States')
nyc = get_latLong('New York City','New York','United States')

In [116]:
print(distance.distance(dc,nyc).miles)

204.67279260883512


In [117]:
# get data
data = pd.read_csv('../data/crm_address.csv')
data.sort_values('cnt',ascending=False,inplace=True)
data['longitude'] = 'N/A'
data['latitude'] = 'N/A'
data['Error'] = 0

In [118]:
#remove unwanted addresses
data = data.loc[(data['state_name'] != 'Alberta')&(data['state_name'] != 'American Samoa')&
                 (data['state_name'] != 'British Columbia')&(data['state_name'] != 'Canal Zone')&
                 (data['state_name'] != 'Federated States of Micronesia')&(data['state_name'] != 'Guam')&
                 (data['state_name'] != 'Marshall Islands')&(data['state_name'] != 'Military Americas')&
                 (data['state_name'] != 'Military Europe')&(data['state_name'] != 'Military Pacific')&
                 (data['state_name'] != 'New Brunswick')&(data['state_name'] != 'Northern Mariana Islands')&
                 (data['state_name'] != 'Northwest Territories')&(data['state_name'] != 'Nova Scotia')&
                 (data['state_name'] != 'Ontario')&(data['state_name'] != 'Prince Edward Island')&
                 (data['state_name'] != 'Quebec')&(data['state_name'] != 'Republic of Palau')&
                 (data['state_name'] != 'Saskatchewan')&(data['state_name'] != 'Virgin Islands')&
                 (data['state_name'] != 'Yukon')]

In [119]:
batch = data.loc[(data['country_name'].notnull()) & (data['state_name'].notnull()) &
                (data['longitude']=='N/A')]
#batch.reset_index(inplace=True)

In [120]:
batch = data.head(100)

In [130]:
for index, row in data.iterrows():
    try:
        lat, long = get_latLong(row.city,row.state_name, row.country_name)
        # calculate distances
        nyc_dist = distance.distance((lat,long),nyc).miles
        dc_dist = distance.distance((lat,long),dc).miles
        
        #append final data
        myData.append([row.city, row.state_name, row.country_name, lat, long, nyc_dist, dc_dist])
        #data['longitude'].iloc[index]=long
        #data['latitude'].iloc[index]=lat
    except TypeError:
        continue

In [131]:
df = pd.DataFrame(myData, columns = ['city','state','country','latitude','longitude',
                                     'nyc_distance','dc_distance'])

In [133]:
df.to_csv('../data/api_distance_results.csv', index=False)

,city,state,country,latitude,longitude,nyc_distance,dc_distance
